In [1]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [2]:
MODEL_NAME = "gpt2-xl" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).cuda()
# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [4]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature=0.7, max_new_tokens=100)

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)

person = "Miles Davis"
texts = [f"{person} has won"]

# texts = [f"List 5 interesting facts about {person}. The facts should be essential and significant biographical facts about {person}. List each fact on its own line and write each as a complete sentence.\n"]

for t in generate_text(texts, model): print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Miles Davis has won nine Grammys, six Oscars, two Golden Globes, two Brits, two Grammy awards, and one Grammy award for best new artist. He's won a total of 10 Grammys. He's done two albums with Yoko Ono. He was the first person to record in New York City on a giant screen. He's the only person to record and play on Broadway. He's the only person to sell a million records, and he's the only person to win a Grammy in


In [28]:
# testing forced generation
def generate_text(texts, model, force_ids):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, num_beams = 20, temperature = .7, repetition_penalty = 10., max_new_tokens=1000, force_words_ids = force_ids)

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)

# person = "Miles Davis"
# texts = [f"{person} has won"]
texts = ["all of which is true of Aretha Franklin"]

# texts = [f"List 5 interesting facts about {person}. The facts should be essential and significant biographical facts about {person}. List each fact on its own line and write each as a complete sentence.\n"]
# force_ids = [tokenizer(["Miles Davis"])["input_ids"]]
force_disj_ids = [tokenizer(["Miles Davis", "Jimi Hendrix"])["input_ids"]]
for t in generate_text(texts, model, force_disj_ids): print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


all of which is true of Aretha Franklin as well.

"She's a great singer, but I don't think she can sing like Beyoncé," he said in an interview with the New York Daily News last year. "I'm not saying that to be mean or anything."

In fact, when asked about his thoughts on her performance at this year's Super Bowl Halftime Show, Trump was more than happy to take credit for it — and even went so far as to say: "Beyoncé did a fantastic job!"

The president-elect has also been known to go out of his way to praise other celebrities who have criticized him over the course of their careers (see: Rosie O'Donnell). He once told Rolling Stone writer Matt Taibbi that Meryl Streep should win an Academy Award for Best Actress every time she opens her mouth ("Meryl Streep gets no respect"). And earlier this month, after The Washington Post reported that Melania Trump had plagiarized portions of Michelle Obama's 2008 Democratic National Convention speech, Donald Trump Jr. took to social media to defen

In [24]:
force_disj_ids = [tokenizer(["Miles Davis", "Jimi Hendrix"])["input_ids"]]

[[44, 2915, 7802], [18050, 72, 14666, 8609]]

In [273]:
def generate_ids_and_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids= model.generate(**encoding, max_new_tokens=10, output_scores = True, return_dict_in_generate=True)

        generated_texts = tokenizer.batch_decode(
            generated_ids.sequences, skip_special_tokens=True
        )
        
    return(generated_ids, generated_texts)

x, gentexts = generate_ids_and_text("Lebron James is perhaps the greatest professional", model)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [274]:
token_id = tokenizer([" basketball", " baseball"])["input_ids"]
token_id

[[9669], [9283]]

In [275]:
from torch.nn import functional as F

s = F.log_softmax(x.scores[-1], -1)
s[0,token_id]


tensor([[3.1579e-01],
        [2.2120e-04]], device='cuda:0')

In [263]:
gentexts

['Lebron James\'s mother, Gloria James, said she was "very proud']